# Data Collection (OMDb + Box Office)

We'll add text and stuff later.

## 1. Setup

In [1]:
import os
import json
import pandas as pd
import requests
import time
import re

base_dir = os.getcwd()
json_path = os.path.join(base_dir, "..", "data_collection", "OMDb", "omdb_data.json")
csv_output_path = os.path.join(base_dir, "..", "data_collection", "OMDb", "omdb_cleaned.csv")

## 2. Fetching OMDb Data

text

see the omdb_fetch.py script

## 3. Load + Explore JSON

text

In [2]:
# into dataframe
with open(json_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head(3)

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,Drama,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",A banker convicted of uxoricide forms a friend...,...,82,9.3,"3,015,278",tt0111161,movie,N/A,"$28,767,189",N/A,N/A,True
1,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan",The aging patriarch of an organized crime dyna...,...,100,9.2,"2,108,592",tt0068646,movie,N/A,"$136,381,073",N/A,N/A,True
2,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart",When a menace known as the Joker wreaks havoc ...,...,84,9.0,"2,997,696",tt0468569,movie,N/A,"$534,987,076",N/A,N/A,True


## 4. Cleaning OMDb Data

text

### 4.1 Selecting relevant columns

In [3]:
columns_to_keep = [
    "Title", "Year", "Genre", "Director", "Actors",
    "Language", "Country", "Runtime", "BoxOffice",
    "imdbRating", "imdbVotes", "Awards", "Type"
]

df = df[columns_to_keep]
df.head(3)

,Title,Year,Genre,Director,Actors,Language,Country,Runtime,BoxOffice,imdbRating,imdbVotes,Awards,Type
0,The Shawshank Redemption,1994,Drama,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",English,United States,142 min,"$28,767,189",9.3,"3,015,278",Nominated for 7 Oscars. 21 wins & 42 nominatio...,movie
1,The Godfather,1972,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan","English, Italian, Latin",United States,175 min,"$136,381,073",9.2,"2,108,592",Won 3 Oscars. 31 wins & 31 nominations total,movie
2,The Dark Knight,2008,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart","English, Mandarin","United States, United Kingdom",152 min,"$534,987,076",9.0,"2,997,696",Won 2 Oscars. 164 wins & 165 nominations total,movie


### 4.2 Cleaning numeric fields

In [4]:
#BoxOffice cleanup
df["BoxOffice"] = df["BoxOffice"].replace("N/A", None)

df["BoxOffice"] = df["BoxOffice"].dropna().apply(
    lambda x: int(re.sub(r"[\$,]", "", x)) if isinstance(x, str) else None
)

#imdbRating cleanup
df["imdbRating"] = pd.to_numeric(df["imdbRating"], errors="coerce")

#imdbVotes cleanup
df["imdbVotes"] = df["imdbVotes"].replace("N/A", None)

df["imdbVotes"] = df["imdbVotes"].dropna().apply(
    lambda x: int(x.replace(",", "")) if isinstance(x, str) else None
)

# saving cleaned data to csv
df.to_csv(csv_output_path, index=False)

## 5. Box Office Missing Values

After cleaning, we check how many movies do not contain valid box office revenue data.

In [5]:
boxoffice_miss = df[df["BoxOffice"].isna()]
print(f"{len(boxoffice_miss)} out of {len(df)} movies are missing BoxOffice data.")

208 out of 988 movies are missing BoxOffice data.


208 movies lack `BoxOffice` data from OMDb. These will be handled by a custom scraper using external sources.

### 5.1 Saving missing movies for scraper

In [6]:
missing_titles = boxoffice_miss["Title"]
missing_path = os.path.join(base_dir, "..", "data_collection", "BoxOfficeScraper", "missing_boxoffice.txt")

missing_titles.to_csv(missing_path, index=False, header=False)

### 5.2 Scraping missing Box Office values

in work

## 6 Final Data

